In [1]:
from ConfigSpace import (
    ConfigurationSpace, Configuration
)
from smac import AlgorithmConfigurationFacade, Scenario
from smac.runhistory.dataclasses import TrialValue

from src.main import *
from copy import deepcopy
import tempfile
from pathlib import Path

from src.portfolio import Portfolio
from src.instance.TSP_Instance import TSP_Instance
from src.solver.TSP_Solver import TSP_Solver
from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import TEMP_DIR

In [2]:
portfolio = Portfolio(size=2, solver_class=TSP_Solver,  configspace=CONFIGURATION_SPACE)
training_instances = [
    TSP_Instance("1.tsp", 20887545.00),
    TSP_Instance("2.tsp", 21134211.00),
    TSP_Instance("3.tsp", 21196547.00),
    TSP_Instance("4.tsp", 21428037.00),
    TSP_Instance("5.tsp", 11020488.00),
]

smac_configuration_space = ConfigurationSpace()
for idx, solver in enumerate(portfolio.solvers):
    for k, v in solver.config.config_space.items():
        v = deepcopy(v)
        v.name = f"{idx}__{v.name}"
        smac_configuration_space.add(v)

In [3]:
t_c = 100
temp_dir = tempfile.TemporaryDirectory(dir=TEMP_DIR)
output_directory = Path(temp_dir.name)

scenario = Scenario(
    configspace=smac_configuration_space, 
    output_directory=output_directory,
)

smac = AlgorithmConfigurationFacade(
    scenario,
    lambda seed: None,
    overwrite=True,
)

while t_c > 0:
    trial_info = smac.ask()
    portfolio.update_config(trial_info.config)
    cost, time = portfolio.evaluate(training_instances)
    trial_value = TrialValue(cost=cost, time=time)
    smac.tell(trial_info, trial_value)
    t_c -= time

temp_dir.cleanup()


[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 460 to 25 (max_ratio == 0.25).


[INFO][abstract_initial_design.py:147] Using 1 initial design configurations and 0 additional configurations.
[INFO][abstract_intensifier.py:516] Added config 9c5d7a as new incumbent because there are no incumbents yet.


In [ ]:
trial_info.config

-41.040000000000006

In [11]:
smac.intensifier.get_incumbent()

Configuration(values={
  '0__ASCENT_CANDIDATES': 50,
  '0__BACKBONE_TRIALS': 0,
  '0__BACKTRACKING': 'NO',
  '0__CANDIDATE_SET_TYPE': 'ALPHA',
  '0__EXTRA_CANDIDATES': 0,
  '0__EXTRA_CANDIDATE_SET_TYPE': 'QUADRANT',
  '0__GAIN23': 'YES',
  '0__GAIN_CRITERION': 'YES',
  '0__INITIAL_STEP_SIZE': 1,
  '0__INITIAL_TOUR_ALGORITHM': 'WALK',
  '0__INITIAL_TOUR_FRACTION': 1.0,
  '0__KICKS': 1,
  '0__KICK_TYPE': 0,
  '0__MAX_BREADTH': 2147483647,
  '0__MAX_CANDIDATES': 5,
  '0__MOVE_TYPE': 5,
  '0__PATCHING_A': 1,
  '0__PATCHING_C': 0,
  '0__POPULATION_SIZE': 50,
  '0__RESTRICTED_SEARCH': 'YES',
  '0__SUBGRADIENT': 'YES',
  '0__SUBSEQUENT_MOVE_TYPE': 0,
  '0__SUBSEQUENT_PATCHING': 'YES',
  '1__ASCENT_CANDIDATES': 50,
  '1__BACKBONE_TRIALS': 0,
  '1__BACKTRACKING': 'NO',
  '1__CANDIDATE_SET_TYPE': 'ALPHA',
  '1__EXTRA_CANDIDATES': 0,
  '1__EXTRA_CANDIDATE_SET_TYPE': 'QUADRANT',
  '1__GAIN23': 'YES',
  '1__GAIN_CRITERION': 'YES',
  '1__INITIAL_STEP_SIZE': 1,
  '1__INITIAL_TOUR_ALGORITHM': 'WALK',


In [5]:
for k, v in smac.intensifier.get_incumbent().items():
    idx = int(k.split("__")[0])
    key = k.split("__")[1]
    portfolio.solvers[idx].config[key] = v
cost, time = portfolio.evaluate(training_instances)
cost, time

(1.6640000000000001, 16.66)

In [13]:
portfolio.solvers[0].solve(training_instances[3])

[<main.TSP_Solver object at 0x000001CFC8EAD750>][16768][2024-11-03 23:48:46.468793] Solving instance 4.tsp
PARAMETER_FILE = config_16768.par
PROBLEM_FILE = 4.tsp
Successes/Runs = 1/1
Cost.min = 21428037, Cost.avg = 21428037.00, Cost.max = 21428037
Gap.min = 0.0000%, Gap.avg = 0.0000%, Gap.max = 0.0000%
Trials.min = 539, Trials.avg = 539.0, Trials.max = 539
Time.min = 2.28 sec., Time.avg = 2.28 sec., Time.max = 2.28 sec.
Time.total = 2.85 sec.
Press any key to continue . . . 


(2.85, 2.85)